In [2]:
import pandas as pd
import datetime
import pickle

GESAMTFILE_b = '/opt/data/optimax_sensing/Gesamtfile_EMA_Baseline_bereinigt_29.01.22_MD.xlsx'
GESAMTFILE_m = '/opt/data/optimax_sensing/Gesamtfile_EMA_MID_29.01.22_MD.xlsx'
GESAMTFILE_p = '/opt/data/optimax_sensing/Gesamtfile_EMA_POST_29.01.22_MD.xlsx'

def create_df(path):
    df = pd.read_excel(path, engine='openpyxl')
    df = df[['VP','Date_Time_last_action (MAX) / completed (SEMA3)','Teilnehmer Mattermost']]

    #Generate placeholder values for missing columns 
    df[['fitbit_id','empatica_id']] = None
    df['platform'] = 'infer'
    df['label'] = df['VP']
    df['end_date'] = None #df['Date_Time_last_action (MAX) / completed (SEMA3)']

    #rename and order columns
    df = df[['Teilnehmer Mattermost','fitbit_id','empatica_id','VP','platform','label','Date_Time_last_action (MAX) / completed (SEMA3)','end_date']]
    df.columns = ['device_id','fitbit_id','empatica_id','pid','platform','label','start_date','end_date']

    #drop NAs in pid column 
    df = df.dropna(subset=['pid'])
    df = df.dropna(subset=['device_id'])

    #Convert dtypes
    df = df.astype({"label": int, "pid": int})
    df['start_date'] = pd.to_datetime(df['start_date'], infer_datetime_format = True)
    #df['start_date'] = df['start_date'].dt.round('D') #round to day, yields inaccurate results 


    #group by participant and get first and last date, put in last two columns 
    end_dates = df.groupby('pid')['start_date'].max().to_frame()
    start_dates = df.groupby('pid')['start_date'].min().to_frame()


    df = df.drop_duplicates(subset=['pid']) #delete duplicate rows 

    df = df.drop(['start_date','end_date'], axis = 1)

    df = df.merge(start_dates, how = 'inner', on = ['pid'])
    df = df.merge(end_dates, how = 'inner', on = ['pid'])

    #rename columns
    df.columns = ['device_id','fitbit_id','empatica_id','pid','platform','label','start_date','end_date']
    return df



In [141]:

df_b = create_df(GESAMTFILE_b)
df_m = create_df(GESAMTFILE_m)
df_p = create_df(GESAMTFILE_p)
gesamt = pd.concat([df_b,df_m,df_p], ignore_index=True)


In [134]:
#gesamt[[gesamt['device_id'].isin({''})]]
gesamt.loc[gesamt['device_id']==' ']

,device_id,fitbit_id,empatica_id,pid,platform,label,start_date,end_date
76,,None,None,453,infer,453,2021-09-04 12:24:59.999999,2021-09-17 18:35:00.000004


In [70]:
#load min-max sensing intervals as extracted from the database (alternatively, use database method to get it)
with open('/home/jmocel/trelium/OPTIMAX-sensing/alltimewindowspertable.pkl', 'rb') as f:
   dizi = pickle.load(f) 

In [95]:
from mappings import RECORD_MAP
tot_interval = pd.DataFrame()

for i in RECORD_MAP:
    tabella =  RECORD_MAP[i]['target_table']
    if tabella in {'APP_USAGE_STATS', 'PHONE_CALENDAR', 'PHONE_RADIO'}:
        continue
    intervals = pd.DataFrame(dizi[tabella], columns=['device_id', 'start','end'])
    tot_interval = pd.concat([tot_interval, intervals], ignore_index=True)    

base_dates = tot_interval.groupby('device_id')['start'].min().to_frame()
end_dates = tot_interval.groupby('device_id')['end'].max().to_frame()


In [96]:
minmax = base_dates.merge(end_dates, how='inner',on=['device_id'])
minmax=minmax.reset_index()

In [102]:
#note how dates for the participant with no name differ between gesamtfile and passive sensing database. Assume they are not the same participant 
res = [i for i in minmax['device_id'] if str(i)[-3:] != '-om']
minmax[minmax['device_id'].isin({''})]


['']

In [142]:
gesamt['device_id'] = [i+'-om' for i in gesamt['device_id']]
part_table = gesamt.loc[gesamt['device_id'].isin(minmax['device_id'])]
part_table = part_table.merge(minmax, how = 'inner', on = ['device_id'])

In [150]:
part_table = part_table.drop(['start_date','end_date'], axis = 1)
part_table.columns = ['device_id', 'fitbit_id', 'empatica_id', 'pid', 'platform', 'label', 'start_date', 'end_date']

In [151]:
part_table.to_csv('/home/jmocel/trelium/OPTIMAX-sensing/AllSensingParticipants.csv')

In [155]:
{str(i) for i in part_table['pid']}

{'102',
 '104',
 '124',
 '131',
 '134',
 '141',
 '149',
 '154',
 '166',
 '170',
 '181',
 '194',
 '201',
 '208',
 '230',
 '255',
 '261',
 '262',
 '263',
 '265',
 '273',
 '284',
 '310',
 '319',
 '326',
 '333',
 '355',
 '366',
 '369',
 '376',
 '383',
 '393',
 '400'}